In [56]:
import json
import numpy as np
import pandas as pd
import re

from pathlib import Path

In [90]:
df = pd.read_json("./results/multinli_transforms_bertscore_dev_matched.jsonl", lines=True)
df = df[df['gold_label'] != '-']
df = df.sample(500)
df.head()

,annotator_labels,genre,gold_label,pairID,promptID,sentence1,sentence1_binary_parse,sentence1_parse,sentence2,sentence2_binary_parse,sentence2_parse,style,sentence1_transform1,sentence1_transform2,sentence2_transform1,sentence2_transform2,sentence1_transform1_bertscore,sentence1_transform2_bertscore,sentence2_transform1_bertscore,sentence2_transform2_bertscore
9714,"[contradiction, contradiction, contradiction, ...",travel,contradiction,124845c,124845,"Since there is no airport on the island, all v...",( ( Since ( there ( is ( ( no airport ) ( on (...,(ROOT (S (SBAR (IN Since) (S (NP (EX there)) (...,The best way to get onto the island is by plane.,( ( ( The ( best way ) ) ( to ( get ( onto ( t...,(ROOT (S (NP (NP (DT The) (JJS best) (NN way))...,emb,Inasmuch as there lacks an aerodrome on this e...,Considering the conspicuous deficit of an airs...,The most efficacious approach to reach the isl...,The preeminent and ideal way to embark upon th...,0.893710,0.877591,0.932928,0.921990
6339,"[entailment, entailment, entailment, entailmen...",travel,entailment,57773e,57773,"Very often the emperor was only a minor, so th...",( Very ( ( often ( the emperor ) ) ( ( ( ( was...,(ROOT (S (ADVP (RB Very)) (NP (RB often) (DT t...,Sometimes the emperor was less than 18 years o...,( Sometimes ( ( the emperor ) ( ( was ( ( ( le...,(ROOT (S (ADVP (RB Sometimes)) (NP (DT the) (N...,sha,"Oft hath the emperor been but a minor, so the ...","The emperor hath been oft a minor, thus the Fu...","On occasion, the emperor bore less than 18 rev...","At times, the emperor was in his youth, having...",0.931644,0.931694,0.879820,0.892570
3385,"[neutral, neutral, neutral, neutral, neutral]",telephone,neutral,109996n,109996,you know Arnold Schwarzenegger is getting to b...,( you ( know ( ( ( ( ( ( Arnold Schwarzenegger...,(ROOT (S (NP (PRP you)) (VP (VBP know) (S (S (...,I prefer Arnold Schwarzenegger as a variety ac...,( I ( ( ( prefer ( ( Arnold Schwarzenegger ) (...,(ROOT (S (NP (PRP I)) (VP (VBP prefer) (NP (NP...,sha,"Verily, thou art aware that Arnold Schwarzeneg...","Hark, heard ye that Arnold Schwarzenegger is e...","Verily, mine eyes prefer Arnold Schwarzenegger...","Pray, I doth prefer Arnold Schwarzenegger as a...",0.879045,0.865861,0.925490,0.915367
1294,"[contradiction, contradiction, contradiction, ...",telephone,contradiction,3476c,3476,apparently apparently the appraisers likes it ...,( apparently ( apparently ( ( ( the appraisers...,(ROOT (S (ADVP (RB apparently)) (ADVP (RB appa...,The appraiser did not like it one bit.,( ( The appraiser ) ( ( ( did not ) ( ( like i...,(ROOT (S (NP (DT The) (NN appraiser)) (VP (VBD...,pir,Apparently the appraisers be likin' it 'cause ...,"Aye, apparently the appraisers be fancyin' it ...",The appraiser did nae fancy it one bit.,The appraiser did not be likin' it one bit.,0.865323,0.865838,0.950855,0.944012
5528,"[entailment, entailment, entailment, entailmen...",slate,entailment,21239e,21239,This call to play fortuneteller is not easily ...,( ( This ( call ( to ( play fortuneteller ) ) ...,(ROOT (S (NP (DT This) (NN call) (S (VP (TO to...,It's not easily refused the call to play fortu...,( It ( ( ( ( 's not ) easily ) ( refused ( the...,(ROOT (S (NP (PRP It)) (VP (VBZ 's) (RB not) (...,sha,This summons to prithee be a soothsayer is not...,This bidding to play the soothsayer is not a t...,The calling to play the seer is not of easy de...,'Tis not an easy task to deny the summons to p...,0.926302,0.935962,0.894918,0.881796


In [94]:
# 1. Remove pairs without gold labels
# 2. Group by transfer style
# 3. Group by genre
# 4. Take 500
# 5. Check if gold labels are balanced

for x, y in df.groupby('genre'):
    for i, j in y.groupby('style'):
        print(len(j))
    print()
# df.head()



34
30
35

37
30
38

37
36
29

33
31
38

31
27
34



In [95]:
for x, y in df.groupby('gold_label'):
    print(len(y))

156
166
178


In [92]:
annotation_set = []

for index, row in df.iterrows():
    premise = row['sentence1_transform1'] if row['sentence1_transform1_bertscore'] > row['sentence1_transform2_bertscore'] else row['sentence1_transform2']
    hypothesis = row['sentence2_transform1'] if row['sentence2_transform1_bertscore'] > row['sentence2_transform2_bertscore'] else row['sentence2_transform2']
    annotation_set.append({
        'pairID': row['pairID'],
        'text': f"Premise:\n{premise}\n\nHypothesis:\n{hypothesis}",
        'label': row['gold_label'],
        'orig_prem': row['sentence1'],
        'orig_hyp': row['sentence2'],
        'orig_label': row['gold_label']
    })

annotation_df = pd.DataFrame(annotation_set[:500])
annotation_df.head()

,pairID,text,label,orig_prem,orig_hyp,orig_label
0,124845c,Premise:\nInasmuch as there lacks an aerodrome...,contradiction,"Since there is no airport on the island, all v...",The best way to get onto the island is by plane.,contradiction
1,57773e,"Premise:\nThe emperor hath been oft a minor, t...",entailment,"Very often the emperor was only a minor, so th...",Sometimes the emperor was less than 18 years o...,entailment
2,109996n,"Premise:\nVerily, thou art aware that Arnold S...",neutral,you know Arnold Schwarzenegger is getting to b...,I prefer Arnold Schwarzenegger as a variety ac...,neutral
3,3476c,"Premise:\nAye, apparently the appraisers be fa...",contradiction,apparently apparently the appraisers likes it ...,The appraiser did not like it one bit.,contradiction
4,21239e,Premise:\nThis bidding to play the soothsayer ...,entailment,This call to play fortuneteller is not easily ...,It's not easily refused the call to play fortu...,entailment


In [93]:
save_path = Path('./results/annotation_subset.json')
annotation_df.to_json(save_path, orient="records")